<a href="https://colab.research.google.com/github/AdrianFernandoPenafiel/03MAIR-Algoritmos-de-optimizacion/blob/master/AG3/AG3_Adrian_Penafiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3 - Actividad Guiada 3

Adrian Peñafiel Schweitzer

